In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize


In [30]:
try:
    df
except NameError:
    df = pd.read_csv("/content/cleanedV2.csv")

# Ensure Skills is a list
def parse_skills(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        parsed = ast.literal_eval(x)
        if isinstance(parsed, list):

            return [str(s).strip().lower() for s in parsed if str(s).strip()]
        return []
    except Exception:

        if isinstance(x, str):
            return [t.strip().lower() for t in x.replace("[","").replace("]","").split(",") if t.strip()]
        return []

df["Skills_List"] = df["Skills"].apply(parse_skills)


df["text"] = df["Cleaned_Resume"].fillna("") + " " + df["Skills_List"].apply(lambda lst: " ".join(lst))


y = df["Category"].astype(str).values
X_text = df["text"].values

X_train, X_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)

clf = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, sublinear_tf=True),
    LinearSVC()
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average="macro")
report = classification_report(y_test, y_pred, output_dict=False)





metrics_df = pd.DataFrame({
    "Metric": ["Accuracy", "F1 (macro)"],
    "Score": [acc, f1_macro]
})
display(metrics_df)
print(report)


student_skills = "python sql machine learning pandas scikit-learn"
predicted_category = clf.predict([student_skills])[0]
print("Predicted Career Path:", predicted_category)

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Metric,Score
0,Accuracy,0.852941
1,F1 (macro),0.794286


                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         2
                     Arts       1.00      1.00      1.00         1
       Automation Testing       0.50      1.00      0.67         1
               Blockchain       1.00      1.00      1.00         1
         Business Analyst       0.50      1.00      0.67         1
           Civil Engineer       1.00      1.00      1.00         1
             Data Science       1.00      1.00      1.00         2
                 Database       1.00      1.00      1.00         2
          DevOps Engineer       1.00      1.00      1.00         1
         DotNet Developer       1.00      1.00      1.00         2
            ETL Developer       1.00      1.00      1.00         1
   Electrical Engineering       1.00      1.00      1.00         1
                       HR       1.00      0.50      0.67         2
                   Hadoop       1.00      1.00      1.00     

In [31]:
vec = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, sublinear_tf=True)
X_all = vec.fit_transform(df["text"].values)

k_candidates = list(range(3, min(12, len(df)//6 + 3)))
sil_scores = []
for k in k_candidates:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labs = km.fit_predict(X_all)
    if len(set(labs)) > 1:
        sil = float(silhouette_score(X_all, labs, metric='cosine'))
    else:
        sil = -1.0
    sil_scores.append(sil)

best_k = int(k_candidates[int(np.argmax(sil_scores))])




kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=25)
cluster_labels = kmeans.fit_predict(X_all)




def top_terms_per_cluster(vec, km, n_terms=13):
    terms = vec.get_feature_names_out()
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    for i in range(km.n_clusters):
        print(f"\nCluster {i}:")
        for ind in order_centroids[i, :n_terms]:
            print("  ", terms[ind])

top_terms_per_cluster(vec, kmeans, n_terms=13)


Cluster 0:
   net
   asp
   dot net
   dot
   asp net
   css
   mvc
   net developer
   html
   sql
   javascript
   developer
   web

Cluster 1:
   sale
   manager
   management
   business
   team
   mba
   customer
   process
   client
   report
   assistant
   work
   sale manager

Cluster 2:
   january
   board
   mumbai
   maharashtra
   university january
   llb
   university
   art
   advocate
   maharashtra state
   state board
   mumbai maharashtra
   state

Cluster 3:
   civil
   civil engineer
   engineer
   construction
   site
   autocad
   site engineer
   civil engineering
   engineer civil
   design engineer
   mumbai
   engineering
   material

Cluster 4:
   exprience less
   less year
   less
   exprience
   year
   year month
   month
   year monthscompany
   matlab
   university skill
   python
   education
   monthscompany

Cluster 5:
   electrical
   engineering
   pune
   day
   panel
   conducted
   power
   diploma
   college
   maharashtra
   mechanical
   p

In [32]:

X_norm = normalize(X_all, norm="l2", axis=1)


nn = NearestNeighbors(metric="cosine", n_neighbors=10)
nn.fit(X_norm)

example_skills = "python, sql, machine learning, pandas, scikit-learn"


vx = vec.transform([example_skills])
vx = normalize(vx, norm="l2", axis=1)


distances, indices = nn.kneighbors(vx, n_neighbors=8)

similarities = 1 - distances.flatten()


recommended_cats = df.iloc[indices.flatten()]["Category"].values


top_categories = (
    pd.Series(recommended_cats)
    .value_counts()
    .head(5)
    .reset_index()
)
top_categories.columns = ["Recommended Category", "Votes"]


print("Example skills:", example_skills)
print("\nTop recommended categories based on similarity:")
print(top_categories)


summary = {
    "best_k": best_k,
    "example_input": example_skills,
    "recommendations": top_categories.to_dict(orient="records"),
}


Example skills: python, sql, machine learning, pandas, scikit-learn

Top recommended categories based on similarity:
  Recommended Category  Votes
0         Data Science      6
1             Database      1
2     Python Developer      1
